In [36]:
import numpy as np
import pydicom 
import matplotlib.pyplot as plt
from  PIL import Image

##### Open one dicom file and disply image

In [89]:
dicom = pydicom.dcmread('./Pantex/DICOM/PAT_0000/STD_0001/SER_0000/OBJ_0001/IM_0001')
print(f"{dicom.PatientID} sex:{dicom.PatientSex} age:{dicom.PatientBirthDate}")
print(f"\n{dicom.get_item([0x0008, 0x1030])}\n") 
# print( dicom.file_meta)
# for i in iter( dicom.iterall()):
#   print(i)
dicom_dict = dicom.to_json_dict()
print(dicom_dict['00181030']['Value'][0])
# /Users/white/Documents/AlexProjects/DICOM_OCC_MED/Pantex/DICOM/PAT_0000/STD_0001/SER_0000/OBJ_0001
# dicom = pydicom.dcmread('./dicom/ILO_Standards/4EF026B7') #read the dicom file into a variable
image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
finale_image = Image.fromarray(final_image_pixel_array) #use image library to convert array of pixel data to an image
finale_image.show() # will open a copy of the image in Preview
# finale_image.save('CXR.PNG') #  Saves to file

P05-0181 sex:M age:19330313

RawDataElement(tag=(0008, 1030), VR='LO', length=12, value=b'CHEST 1 VIEW', value_tell=700, is_implicit_VR=False, is_little_endian=True, is_raw=True)

Chest AP


In [20]:
root_dir = './Pantex/Profusion Dicoms/'  #'./Pantex/Profusion Dicoms/'
 

##### Find all the types of files stored in our dicom files 

In [4]:
# 
import glob
study_types = [] 
path_to_match = f'{root_dir}**/IM*' #ie,    './Pantex/**/IM*'
for f in glob.glob(path_to_match, recursive=True):
  dicom = pydicom.dcmread(f)
  study_description = str( dicom.get_item([0x0008, 0x1030]).value)
  #Find is the image is an X-ray or a different type of image (like a scanned ILO form)
  image_type = "NOT X-RAY"
  try:
    dicom_dict = dicom.to_json_dict()
    image_type=dicom_dict['00181030']['Value'][0] # Will have value like 'CHEST PA' or 'CHEST LAT', we are only interested in AP and PA films
    study_types.append(image_type)
  except Exception:
    pass
list(set( study_types))



['CHEST']

In [32]:
import glob
path_to_match = f'{root_dir}**/IM*' #ie,    './Pantex/**/IM*'
for f in glob.glob(path_to_match, recursive=True):
  dicom = pydicom.dcmread(f)
  study_description = str( dicom.get_item([0x0008, 0x1030]).value)
  #Find is the image is an X-ray or a different type of image (like a scanned ILO form)
  image_type = "NOT X-RAY"
  try:
    dicom_dict = dicom.to_json_dict()
    image_type=dicom_dict['00181030']['Value'][0] # Will have value like 'CHEST PA' or 'CHEST LAT', we are only interested in AP and PA films
  
    # if((image_type !='Chest LAT') and (image_type !='NOT X-RAY') ):
      # ANONYMIZE, JUST IN CASE (should already be stripped)
    dicom.PatientName = None # 'ANON'
    dicom.PatientBirthDate = None #'19000101'
    PA_or_AP = 'AP' if image_type =='Chest AP' else 'PA'
    filename = f"{dicom.PatientID}_{dicom.StudyDate}_{PA_or_AP}.dcm"
    png_filename = f"{dicom.PatientID}_{dicom.StudyDate}.png"
    print(f,png_filename)
    dicom.save_as(f'{root_dir}FLATTENED/{filename}')

    image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
    print(f'm/m {image_pixel_array.max()} {image_pixel_array.min()}')
    # rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
    rescaled_image_pixel_array = np.interp(image_pixel_array, (image_pixel_array.min(),image_pixel_array.max()),(0,255))
    final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
    finale_image = Image.fromarray(final_image_pixel_array) #use image library to convert array of pixel data to an image
    # finale_image.show() # will open a copy of the image in Preview
    finale_image.save(f'{root_dir}FLATTENED/{png_filename}') #  Saves to file
  except Exception:
    pass

    # files.append([f,study_description,image_type])
    # image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
    # rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
    # final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
    # finale_image = Image.fromarray(final_image_pixel_array) #use image library to convert array of pixel data to an image
    # finale_image.show()
# len(files)

  

./Pantex/Profusion Dicoms/DICOM/PAT_0011/STD_0001/SER_0000/OBJ_0001/IM_0001 P13-0043_20131010.png
m/m 4055.0 68.0
./Pantex/Profusion Dicoms/DICOM/PAT_0011/STD_0001/SER_0001/OBJ_0001/IM_0001 P13-0043_20131010.png
m/m 4089.0 0.0
./Pantex/Profusion Dicoms/DICOM/PAT_0016/STD_0000/SER_0000/OBJ_0001/IM_0001 P06-0172_20110901.png
m/m 4051.0 14.0
./Pantex/Profusion Dicoms/DICOM/PAT_0017/STD_0003/SER_0000/OBJ_0001/IM_0001 P10-0066_20130725.png
m/m 3992.0 42.0
./Pantex/Profusion Dicoms/DICOM/PAT_0017/STD_0002/SER_0000/OBJ_0001/IM_0001 P10-0066_20100630.png
m/m 3611.0 40.0
./Pantex/Profusion Dicoms/DICOM/PAT_0017/STD_0002/SER_0000/OBJ_0001/IM_0002 P10-0066_20100630.png
m/m 4092.0 0.0
./Pantex/Profusion Dicoms/DICOM/PAT_0010/STD_0001/SER_0000/OBJ_0001/IM_0001 P13-0046_20131017.png
m/m 3910.0 56.0
./Pantex/Profusion Dicoms/DICOM/PAT_0010/STD_0001/SER_0001/OBJ_0001/IM_0001 P13-0046_20131017.png
m/m 3962.0 0.0
./Pantex/Profusion Dicoms/DICOM/PAT_0003/STD_0001/SER_0000/OBJ_0001/IM_0001 P05-0157_201505

### Crop pixel array to remove labels

In [57]:
  file = "./Pantex/Profusion Dicoms/FLATTENED/P09-0101_20211209_PA.dcm"
  dicom = pydicom.dcmread(file)
 

  image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
  image_pixel_array = dicom.pixel_array[150:2500][120:3000]
  # print(f'm/m {image_pixel_array.max()} {image_pixel_array.min()}')
  # # np.histogram(image_pixel_array)
  # plt.hist(image_pixel_array)





In [56]:
  old_range =  (image_pixel_array.min(),image_pixel_array.max()) if True else  (image_pixel_array.max(),image_pixel_array.min()) 
  # rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
  rescaled_image_pixel_array = np.interp(image_pixel_array, old_range,(0,255))
  final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
  # print(final_image_pixel_array.shape())
  # cropped_img_arr = final_image_pixel_array[160:2500][120:3000]
  finale_image = Image.fromarray(cropped_img_arr) #use image library to convert array of pixel data to an image
  finale_image.show() # will open a copy of the image in Preview
  # finale_image.save(f'{root_dir}FLATTENED/{png_filename}') #  Saves to file

np.shape(final_image_pixel_array)

# pretty print all the files in an direcory
# print('\n'.join(os.listdir('./Pantex/Profusion Dicoms/FLATTENED/recheck')))

(2220, 3072)